In [10]:
import argparse
import torch
import math
import numpy as np
import time
import pickle 
from torch.nn.functional import gumbel_softmax
from utils import gumbel_softmax_3d

In [27]:
class SKModel():
    def __init__(self, n=20, seed=0, device='cuda:0'):
        self.n = n
        self.seed = seed
        self.device = device
        if self.seed > 0:
            torch.manual_seed(self.seed)
        self.J = torch.randn(self.n, self.n) / math.sqrt(self.n)
        self.J = torch.triu(self.J, diagonal=1)
        self.J = self.J + self.J.t()
        self.J_np = self.J.numpy()

    def gumbel_optimization(self, bs=128, max_iters=20000, lr=1, eta=1e-3, init_tau=20, final_tau=1, diff=1e-8,elim_rate=0.125,mut_rate=1e-3):
        n = self.n
        device = self.device
        J = self.J.to(device)
        # learnable parameters
        # x = torch.randn(bs, n, 1, device=device, requires_grad=True)
        x = torch.randn(bs, n, 1, device=device) * 1e-5
        x.requires_grad = True
        optimizer = torch.optim.Adam([x], lr=lr)
        tau = init_tau
        decay = (init_tau - final_tau) / max_iters
        E_best = torch.ones(1, device=device)

        for i in range(max_iters):
            E_old = E_best.clone()
            optimizer.zero_grad()
            probs = torch.empty(bs, n, 2, device=device)
            p = torch.sigmoid(x)
            probs[:, :, 0] = p.squeeze()
            probs[:, :, -1] = 1 - probs[:, :, 0]
            logits = torch.log(probs + 1e-10)
            s = 2 * gumbel_softmax_3d(logits, tau=tau, hard=False)[:, :, 0] - 1
            E = -0.5 * torch.sum((s @ J) * s, dim=1) / n
            constraint = torch.sum(x ** 2) / n / bs
            loss = torch.mean(E) + eta * constraint
            optimizer = torch.optim.Adam([x], lr=lr)
            loss.backward()
            optimizer.step()
            tau -= decay
            with torch.no_grad():
                if i % 100 == 0:
                    print(i,'Current best result: %.8f' % (E_best.cpu().numpy()))
                s = 2 * gumbel_softmax_3d(logits, tau=tau, hard=True)[:, :, 0] - 1
                E = -0.5 * torch.sum((s @ J) * s, dim=1) / n
                Emin = torch.min(E)
                Emean = torch.mean(E)
                if i % 100 == 0:
                    E_np = E.cpu().numpy()
                    E_np_min = np.min(E_np)
                    E_sorted=np.sort(E_np)
                    index_sorted=np.argsort(E_np)
                    print('Energy_sorted:\n', E_sorted[0:16])
                    print('label_sorted:\n', index_sorted[0:16])
                    for i in index_sorted[0:16]:
                        print(i)
                        print(s[i,:])
                if Emin < E_best:
                    E_best = Emin
                
                ##############Evolution#############
                if i % 1000000000000 == 0 and E_old - Emin < 1e-6:
                    
                    
                    E_np = E.cpu().numpy()
                    E_sorted=np.sort(E_np)
                    index_sorted=np.argsort(E_np)
                    #print('Energy_sorted:\n', E_sorted)
                    #print('label_sorted:\n', index_sorted)
                    
                    
                    w = np.max(E_np) - E_np
                    fitness = w / np.sum(w)
                    
                    
                    #print('fitness:\n', fitness)
                    ac = 0
                    f = np.empty(bs)
                    #print('roulette:')
                    for i in range(bs):
                        f[i] = ac + fitness[i]
                        ac = f[i]
                        #print(i, f[i])
                    r = np.random.random()
                    
                    
                    #print('\nGA.select1:')
                    j = 0
                    z = np.ones(bs, dtype=int)
                    z = z * (bs + 1)
                    while j < bs * (1 - elim_rate):
                        r = np.random.random()
                        if 0 <= r < f[0]:
                            if 0 not in z:
                                z[j] = 0
                                j = j + 1
                                continue
                        elif f[bs - 1] <= r <= 1:
                            if (bs - 1) not in z:
                                z[j] = (bs - 1)
                                j = j + 1
                                continue
                        else:
                            for t in range(bs):
                                if f[t] <= r < f[t + 1]:
                                    if t not in z:
                                        z[j] = t
                                        j = j + 1
                                        break    
                    #print(z)
                    
                    
                    #print('\nGA.select2:')
                    k = 0
                    elim = int(bs * elim_rate)
                    u = np.ones(elim,dtype=int)
                    while k < bs * elim_rate:
                        q = np.random.random()
                        if 0 <= q < f[0]:
                            if 0 in z:                
                                u[k] = 0
                                k = k + 1
                                continue
                        elif f[bs - 1] <= q <= 1:
                            if (bs-1) in z:
                                u[k] = (bs - 1)
                                k = k + 1
                                continue
                        else:
                            for o in range(bs):
                                if f[o] <= q < f[o + 1]:
                                    if o in z:
                                        u[k] = o
                                        k = k + 1
                                        break
                    u = u.reshape(int(elim/2),2)
                    #print(u)
                    
                    
                    #print('\nGA.crossover:')
                    x_ = torch.empty(bs,n,1,device=device)
                    x_.requires_grad = True
                    for l in range(int(elim/2)):
                        n1 = u[l,0]
                        n2 = u[l,1]
                        x1 = x[n1,:,0].split(int(n/2))
                        x2 = x[n2,:,0].split(int(n/2))
                        x11 = x1[0]
                        x12 = x1[1]
                        x21 = x2[0]
                        x22 = x2[1]
                        x_1 = torch.cat((x11,x22))
                        x_2 = torch.cat((x21,x12))
                        x_[int(bs*(1-elim_rate))+2*l,:,0]=x_1
                        x_[int(bs*(1-elim_rate))+2*l+1,:,0]=x_2
                        for i in range(int(bs * (1 - elim_rate))):
                            x_[i,:,0]=x[z[i],:,0]
                    #print(x.size())
                    #print(x_.size())

                    
                    #print('\nGA.mutation:')
                    mut_count = 0
                    for i in range(bs):
                        for j in range(n):
                            if np.random.random() <= mut_rate:
                                x_[i,j,0] = torch.randn(1,device=device) * 1e-5
                                mut_count += 1
                    #print(mut_count)
                    
                    
                    #p_ = torch.sigmoid(x_)
                    #probs_ = torch.empty(bs, n, 2, device=device)
                    #probs_[:, :, 0] = p_.squeeze()
                    #probs_[:, :, -1] = 1 - probs_[:, :, 0]
                    #logits_ = torch.log(probs_ + 1e-10)
                    #s_ = 2 * gumbel_softmax_3d(logits_, tau=tau, hard=False)[:, :, 0] - 1
                    #E_ = -0.5 * torch.sum((s_ @ J) * s_, dim=1) / n
                    #if torch.mean(E_)<Emean or torch.min(E_)<Emin:
                        x=x_
                    ###################################
                    
                                        
                if torch.abs(Emin - E_old) < diff:
                    break
        return E_best.cpu().numpy()

In [28]:
def main():
    # settings
    parser = argparse.ArgumentParser('Gumbel-softmax optimizing SK model energy')
    parser.add_argument('--n', type=int, default=1024,
                        help='size (default: 1024)')
    parser.add_argument('--bs', type=int, default=128,
                        help='batch size (default: 128)')
    parser.add_argument('--max_iters', type=int, default=50000,
                        help='iterations (default: 50000)')
    parser.add_argument('--lr', type=float, default=1.,
                        help='learning rate (default: 1)')
    parser.add_argument('--eta', type=float, default=1e-3,
                        help='weight decay (default: 1e-3)')
    parser.add_argument('--init-tau', type=float, default=20.,
                        help='initial tau in Gumbel-softmax (default: 20)')
    parser.add_argument('--final-tau', type=float, default=1.,
                        help='final tau in Gumbel-softmax (default: 1)')
    parser.add_argument('--instances', type=int, default=100,
                        help='number of ensembles (default: 1024)')
    parser.add_argument('--device', type=str, default='cuda:0',
                        help='cuda device (default: cuda:0)')
    args = parser.parse_args(args=[])
    n = args.n  # size
    bs = args.bs # batch size
    max_iters = args.max_iters
    init_tau = args.init_tau
    final_tau = args.final_tau
    lr = args.lr
    eta = args.eta
    instances = args.instances
    device = args.device
    torch.manual_seed(2050)

    # training
    results_arr = []
    for i in range(instances):
        sk = SKModel(n, device=device)
        energy = sk.gumbel_optimization(bs, max_iters, lr, eta, init_tau, final_tau)
        print('# %i \t energy: %.5f' % (i, energy))
        results_arr.append(energy)

   # print mean energy and std
    data = np.array(results_arr)
    data_mean = data.mean()
    data_std = np.sqrt(np.var(data, ddof=1))
    print('Batch Gumbel-softmax N: %i, bs: %i, instances: %i, mean: %.5f, std: %.5f' %
          (n, bs, instances, data_mean, data_std))


if __name__ == '__main__':
    start = time.time()
    main()
    end = time.time()
    print('Running time: %.5f s \n' % (end - start))


0 Current best result: 1.00000000
Energy_sorted:
 [-0.05446712 -0.05437486 -0.04368582 -0.04135018 -0.03982198 -0.03951745
 -0.03733383 -0.03646356 -0.03508395 -0.03255935 -0.03253445 -0.03036683
 -0.02973241 -0.02964533 -0.02947187 -0.02903254]
label_sorted:
 [ 99  32  84  45 107 124  56  72  29  24  98  28 114  91 121  96]
99
tensor([ 1., -1.,  1.,  ..., -1., -1.,  1.], device='cuda:0')
32
tensor([-1., -1., -1.,  ..., -1., -1.,  1.], device='cuda:0')
84
tensor([-1., -1.,  1.,  ..., -1., -1.,  1.], device='cuda:0')
45
tensor([-1., -1., -1.,  ...,  1., -1., -1.], device='cuda:0')
107
tensor([-1.,  1.,  1.,  ..., -1., -1.,  1.], device='cuda:0')
124
tensor([-1., -1., -1.,  ..., -1., -1.,  1.], device='cuda:0')
56
tensor([ 1., -1.,  1.,  ...,  1.,  1.,  1.], device='cuda:0')
72
tensor([ 1., -1.,  1.,  ..., -1., -1.,  1.], device='cuda:0')
29
tensor([ 1., -1., -1.,  ...,  1.,  1., -1.], device='cuda:0')
24
tensor([-1.,  1.,  1.,  ...,  1.,  1.,  1.], device='cuda:0')
98
tensor([ 1.,  1., 

600 Current best result: -0.09390767
Energy_sorted:
 [-0.05055113 -0.04664255 -0.04505819 -0.0416052  -0.03941452 -0.03926935
 -0.03660139 -0.03607442 -0.03585934 -0.03520074 -0.03120151 -0.02981333
 -0.02824805 -0.0270385  -0.026391   -0.02541035]
label_sorted:
 [  7   4   8  34  60  49  86 108 113 107 112  35  98  74  11  36]
7
tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0')
4
tensor([-1.,  1.,  1.,  ..., -1., -1., -1.], device='cuda:0')
8
tensor([-1.,  1., -1.,  ...,  1.,  1.,  1.], device='cuda:0')
34
tensor([ 1., -1., -1.,  ..., -1.,  1.,  1.], device='cuda:0')
60
tensor([-1., -1., -1.,  ..., -1., -1., -1.], device='cuda:0')
49
tensor([-1., -1.,  1.,  ...,  1.,  1.,  1.], device='cuda:0')
86
tensor([-1., -1.,  1.,  ..., -1., -1., -1.], device='cuda:0')
108
tensor([-1., -1.,  1.,  ..., -1., -1.,  1.], device='cuda:0')
113
tensor([-1., -1.,  1.,  ..., -1.,  1.,  1.], device='cuda:0')
107
tensor([-1.,  1., -1.,  ...,  1.,  1.,  1.], device='cuda:0')
112
tensor([-1.,  1., -1.,

1200 Current best result: -0.11196916
Energy_sorted:
 [-0.06089921 -0.05303711 -0.0482222  -0.0413551  -0.03756249 -0.03708905
 -0.03619216 -0.03465541 -0.03000462 -0.02964039 -0.02891332 -0.02803484
 -0.02762731 -0.02593729 -0.02517117 -0.02167479]
label_sorted:
 [ 38 122  52  73  71  43 120 118 108  68 121  24  33   0  81 125]
38
tensor([-1., -1.,  1.,  ..., -1.,  1., -1.], device='cuda:0')
122
tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0')
52
tensor([ 1.,  1., -1.,  ..., -1., -1.,  1.], device='cuda:0')
73
tensor([ 1.,  1., -1.,  ..., -1., -1., -1.], device='cuda:0')
71
tensor([-1., -1.,  1.,  ..., -1.,  1.,  1.], device='cuda:0')
43
tensor([ 1.,  1.,  1.,  ...,  1., -1., -1.], device='cuda:0')
120
tensor([ 1., -1.,  1.,  ...,  1.,  1.,  1.], device='cuda:0')
118
tensor([ 1., -1.,  1.,  ...,  1.,  1.,  1.], device='cuda:0')
108
tensor([ 1., -1.,  1.,  ...,  1., -1., -1.], device='cuda:0')
68
tensor([-1.,  1.,  1.,  ...,  1.,  1., -1.], device='cuda:0')
121
tensor([ 1., -1.,

1800 Current best result: -0.11196916
Energy_sorted:
 [-0.07072246 -0.04942395 -0.04906658 -0.04904611 -0.04809102 -0.04585243
 -0.04463246 -0.04379202 -0.04084018 -0.04042334 -0.03864724 -0.03758626
 -0.03697743 -0.03451365 -0.03446042 -0.03292744]
label_sorted:
 [119  71  24   6  56 120  23  92  95 126  85  44  15 116  76  35]
119
tensor([-1., -1.,  1.,  ..., -1., -1.,  1.], device='cuda:0')
71
tensor([ 1.,  1.,  1.,  ..., -1.,  1., -1.], device='cuda:0')
24
tensor([-1., -1., -1.,  ...,  1.,  1., -1.], device='cuda:0')
6
tensor([-1.,  1., -1.,  ...,  1., -1., -1.], device='cuda:0')
56
tensor([ 1., -1.,  1.,  ..., -1., -1.,  1.], device='cuda:0')
120
tensor([-1., -1.,  1.,  ..., -1.,  1., -1.], device='cuda:0')
23
tensor([-1.,  1., -1.,  ...,  1., -1.,  1.], device='cuda:0')
92
tensor([-1., -1., -1.,  ..., -1., -1., -1.], device='cuda:0')
95
tensor([-1.,  1., -1.,  ..., -1., -1., -1.], device='cuda:0')
126
tensor([-1., -1., -1.,  ...,  1., -1., -1.], device='cuda:0')
85
tensor([ 1.,  

2400 Current best result: -0.11196916
Energy_sorted:
 [-0.04859304 -0.04592847 -0.04580831 -0.04086587 -0.04062319 -0.03834932
 -0.037163   -0.03527387 -0.03495    -0.03472264 -0.0320715  -0.03168369
 -0.03054423 -0.02827662 -0.02798428 -0.02798016]
label_sorted:
 [127  27 122  71  17  10 103  14 125  81  94  15   2  38  95  77]
127
tensor([ 1., -1.,  1.,  ...,  1.,  1.,  1.], device='cuda:0')
27
tensor([ 1., -1.,  1.,  ..., -1.,  1.,  1.], device='cuda:0')
122
tensor([ 1., -1., -1.,  ...,  1., -1.,  1.], device='cuda:0')
71
tensor([ 1.,  1.,  1.,  ...,  1.,  1., -1.], device='cuda:0')
17
tensor([-1., -1.,  1.,  ...,  1., -1., -1.], device='cuda:0')
10
tensor([-1.,  1., -1.,  ...,  1.,  1.,  1.], device='cuda:0')
103
tensor([ 1., -1.,  1.,  ..., -1.,  1.,  1.], device='cuda:0')
14
tensor([ 1., -1.,  1.,  ..., -1., -1.,  1.], device='cuda:0')
125
tensor([ 1.,  1.,  1.,  ..., -1., -1., -1.], device='cuda:0')
81
tensor([ 1., -1.,  1.,  ...,  1.,  1., -1.], device='cuda:0')
94
tensor([ 1.,

...

22800 Current best result: -0.73441207
Energy_sorted:
 [-0.730759   -0.7289861  -0.7286353  -0.72857773 -0.72826475 -0.7275964
 -0.7275895  -0.72752    -0.72739434 -0.7268245  -0.72631377 -0.7262311
 -0.72535837 -0.72528553 -0.725036   -0.7247163 ]
label_sorted:
 [ 95  82 121 124 111  92  48  35  21 125  97 100  29  17  73  53]
95
tensor([ 1., -1., -1.,  ...,  1., -1.,  1.], device='cuda:0')
82
tensor([-1.,  1., -1.,  ..., -1.,  1., -1.], device='cuda:0')
121
tensor([ 1., -1.,  1.,  ..., -1.,  1., -1.], device='cuda:0')
124
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
111
tensor([-1.,  1., -1.,  ..., -1.,  1., -1.], device='cuda:0')
92
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
48
tensor([-1.,  1., -1.,  ..., -1.,  1., -1.], device='cuda:0')
35
tensor([-1.,  1., -1.,  ..., -1.,  1., -1.], device='cuda:0')
21
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
125
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
97
tensor([ 1., 

23400 Current best result: -0.73540926
Energy_sorted:
 [-0.7329881  -0.7325709  -0.730409   -0.7298135  -0.7293953  -0.72930247
 -0.7292799  -0.7289591  -0.72895575 -0.7280092  -0.7279005  -0.72759706
 -0.7273009  -0.7271998  -0.7271246  -0.7267824 ]
label_sorted:
 [125  73  29  21  95  30   6  92  53  48  17 121 100 124 111  97]
125
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
73
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
29
tensor([-1.,  1., -1.,  ..., -1.,  1., -1.], device='cuda:0')
21
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
95
tensor([ 1., -1., -1.,  ...,  1., -1.,  1.], device='cuda:0')
30
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
6
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
92
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
53
tensor([-1.,  1., -1.,  ..., -1.,  1., -1.], device='cuda:0')
48
tensor([-1.,  1., -1.,  ..., -1.,  1., -1.], device='cuda:0')
17
tensor([-1.,  1

24000 Current best result: -0.73702228
Energy_sorted:
 [-0.73408246 -0.73309064 -0.7325252  -0.73198634 -0.73194057 -0.73094034
 -0.73054636 -0.7300893  -0.72988987 -0.7294966  -0.7290735  -0.7286604
 -0.7283404  -0.7280887  -0.727984   -0.72768295]
label_sorted:
 [ 29 124  95 125  73 121   6  65  97  93 106  54 100  63  30  48]
29
tensor([-1.,  1., -1.,  ..., -1.,  1., -1.], device='cuda:0')
124
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
95
tensor([ 1., -1., -1.,  ...,  1., -1.,  1.], device='cuda:0')
125
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
73
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
121
tensor([ 1., -1.,  1.,  ..., -1.,  1., -1.], device='cuda:0')
6
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
65
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
97
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
93
tensor([-1.,  1.,  1.,  ..., -1.,  1., -1.], device='cuda:0')
106
tensor([ 1., 

24600 Current best result: -0.73762000
Energy_sorted:
 [-0.7362707  -0.73387134 -0.7326046  -0.7321676  -0.73193204 -0.7313939
 -0.73137337 -0.7311963  -0.7308854  -0.73080546 -0.7305394  -0.7303907
 -0.7301855  -0.7300195  -0.72998536 -0.7298237 ]
label_sorted:
 [ 95 121  29 110 125  82  48 124 100  61   1  12  73  35  31  21]
95
tensor([ 1., -1., -1.,  ...,  1., -1.,  1.], device='cuda:0')
121
tensor([ 1., -1.,  1.,  ..., -1.,  1., -1.], device='cuda:0')
29
tensor([-1.,  1., -1.,  ..., -1.,  1., -1.], device='cuda:0')
110
tensor([-1.,  1.,  1.,  ..., -1.,  1., -1.], device='cuda:0')
125
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
82
tensor([-1.,  1., -1.,  ..., -1.,  1., -1.], device='cuda:0')
48
tensor([-1.,  1., -1.,  ..., -1.,  1., -1.], device='cuda:0')
124
tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.], device='cuda:0')
100
tensor([-1.,  1., -1.,  ..., -1.,  1., -1.], device='cuda:0')
61
tensor([ 1., -1., -1.,  ...,  1., -1.,  1.], device='cuda:0')
1
tensor([ 1., 

KeyboardInterrupt: 